In [2]:
import cv2

In [85]:
# Image Enhancing

img = cv2.imread('./0_100.jpg')

# gray scale
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# enhance contrast
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(16,16))
equalized_image = clahe.apply(gray)

# Gaussian blur
blurred_image = cv2.GaussianBlur(equalized_image, (5, 5), 0)

# Canny edge
edges = cv2.Canny(blurred_image, threshold1=100, threshold2=250)

cv2.imshow('image',edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

(< cv2.KeyPoint 000001BF66A05D50>, < cv2.KeyPoint 000001BF66A05DE0>, < cv2.KeyPoint 000001BF66A05120>, < cv2.KeyPoint 000001BF66A05780>, < cv2.KeyPoint 000001BF66A05E40>, < cv2.KeyPoint 000001BF66A052D0>, < cv2.KeyPoint 000001BF66A058D0>, < cv2.KeyPoint 000001BF66A054E0>, < cv2.KeyPoint 000001BF66A05AB0>, < cv2.KeyPoint 000001BF66A05690>)
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
